In [1]:
import pandas as pd

In [34]:
df_enem = pd.read_parquet("../data/treated/df_enem.parquet")

Retirando colunas desnecessárias

In [35]:
colums_drop = ['TP_PRESENCA_CN',
'TP_PRESENCA_CH',
'TP_PRESENCA_LC',
'TP_PRESENCA_MT',
'CO_PROVA_CH',
'CO_PROVA_LC',
'CO_PROVA_MT',
'TX_RESPOSTAS_CN',
'TX_RESPOSTAS_CH',
'TX_RESPOSTAS_LC',
'TX_RESPOSTAS_MT',
'TX_GABARITO_CN',
'TX_GABARITO_CH',
'TX_GABARITO_LC',
'TX_GABARITO_MT',
'CO_MUNICIPIO_ESC',
'CO_UF_ESC',
'NU_ANO',
'CO_MUNICIPIO_PROVA',
'CO_UF_PROVA',
'TP_SIT_FUNC_ESC',
'CO_PROVA_CN',
'TP_ST_CONCLUSAO']

#Removendo colunas

df_enem.drop(columns=colums_drop, inplace=True)

Renomeando as colunas para que o modelo tenha um melhor contexto. 

In [2]:
import json
from transformers import pipeline

c:\Users\Marcela Beatriz\infnet_projeto_bloco_2024s2\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
with open('../docs/data_docs/dicionario_contexto.json', 'r') as f:
    dicionario = json.load(f)

In [2]:
df_metrics = pd.read_parquet("../data/analysis/df_uf_region_full.parquet")

In [11]:
df_metrics.head()

,UF,Total Inscritos,Regiao,Total Presentes,Total Homens Inscritos,Total Mulheres Inscritas,Total Homens Presentes,Total Mulheres Presentes,Percentual de Presentes,Total Inscritos Raça Não Declarada,...,Inscritos entre 51 e 55 anos,Inscritos entre 56 e 60 anos,Inscritos entre 61 e 65 anos,Inscritos entre 66 e 70 anos,Inscritos maiores de 70 anos,Inscritos Não Declarados,Inscritos Brasileiros,Inscritos Brasileiros Naturalizados,Inscritos Estrangeiros,Inscritos Brasileiros Natos Nascidos no Exterior
0,DF,72975,Centro-Oeste,50267,28782,44193,19719,30548,68.88,1171,...,708,402,151,65,29,32,71637,1083,104,119
1,RS,159919,Sul,110735,59692,100227,41633,69102,69.24,2087,...,1171,664,304,96,35,41,156812,2520,344,202
2,CE,241960,Nordeste,173700,106239,135721,74852,98848,71.79,5198,...,1084,615,248,99,38,171,237830,3571,201,187
3,BA,324268,Nordeste,220257,113310,210958,77583,142674,67.92,4325,...,2171,1159,449,191,69,182,315972,7471,184,459
4,PE,218859,Nordeste,152574,86048,132811,59547,93027,69.71,2847,...,1486,762,322,120,34,133,213591,4720,103,312


In [29]:
# Função para gerar um texto estruturado a partir do DataFrame
def dataframe_para_texto_estruturado(df):
    texto = "Aqui estão os dados do ENEM 2023:\n"
    for _, row in df.iterrows():
        texto += f"Estado: {row['UF']}\n"
        texto += f"  - Total de Inscritos: {row['Total Inscritos']}\n"
        texto += f"  - Total de Mulheres Inscritas: {row['Total Mulheres Inscritas']}\n"
    return texto




In [47]:
from langchain_community.llms import FakeListLLM, HuggingFaceHub
#from langchain_openai.chat_models import ChatOpenAI
from langchain.schema import HumanMessage
from langchain_core.prompts import ChatPromptTemplate
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_huggingface import HuggingFaceEndpoint, ChatHuggingFace
import os
from dotenv import load_dotenv

In [ ]:
load_dotenv(caminho_env)

# Função para gerar o texto com base na UF
def gerar_texto_uf(uf, df):
    
    # Filtrar dados para a UF informada
    dados_uf = df[df['UF'] == (uf).upper()].iloc[0]
    
    # Gerar o contexto com base nas informações
    contexto = f"Dados do ENEM 2023 para o estado de {uf} em até 2 parágrafos:\n"
    contexto += f" - Total de inscritos {dados_uf['Total Inscritos']}\n"
    contexto += f" - Total de mulheres inscritas {dados_uf['Total Mulheres Inscritas']}\n"
    contexto += f" - Total de mulheres presentes {dados_uf['Total Mulheres Presentes']}\n"
    contexto += f" - Total de homens inscritos {dados_uf['Total Homens Inscritos']}\n"
    contexto += f" - Total de homens presentes {dados_uf['Total Homens Presentes']}\n"


    template = ChatPromptTemplate([
        ("system", "Você é um criador de textos de até dois parágrafos sobre os dados do enem 2023. Caso você não tenha informações, avise ao usuário."),
        ("user", " {contexto}")
    ])

    key = os.getenv('GOOGLE_API_KEY')
    llm = ChatGoogleGenerativeAI(model="gemini-1.5-pro", temperature=0.5)
    response = llm.invoke(template.format_messages(contexto=contexto))
    return response.content
    
uf_input = "rn"  # Estado de entrada
texto_gerado = gerar_texto_uf(uf_input, df_metrics)


No Rio Grande do Norte, o ENEM 2023 contou com um total de 100.706 inscritos, dos quais 59.334 eram mulheres e 41.372 homens.  A presença feminina foi mais expressiva, com 42.868 participantes, enquanto 29.937 homens compareceram às provas.  Isso demonstra uma diferença significativa na participação entre os gêneros, com maior representatividade feminina tanto em número de inscrições quanto de presença.

A taxa de abstenção no estado foi considerável.  Calculando a partir dos dados fornecidos, 17.899 mulheres e 11.435 homens inscritos não compareceram ao exame, totalizando 29.334 ausentes.  Esses números reforçam a importância de investigar os fatores que contribuem para a abstenção no ENEM, visando a inclusão e a igualdade de oportunidades para todos os estudantes.



In [43]:
df_metrics.columns

Index(['UF', 'Total Inscritos', 'Regiao', 'Total Presentes',
       'Total Homens Inscritos', 'Total Mulheres Inscritas',
       'Total Homens Presentes', 'Total Mulheres Presentes',
       'Percentual de Presentes', 'Total Inscritos Raça Não Declarada',
       'Total Inscritos Raça Branca', 'Total Inscritos Raça Preta',
       'Total Inscritos Raça Parda', 'Total Inscritos Raça Amarela',
       'Total Inscritos Raça Indígena', 'Inscritos menores de 17 anos',
       'Inscritos com 17 anos', 'Inscritos com 18 anos',
       'Inscritos com 19 anos', 'Inscritos com 20 anos',
       'Inscritos com 21 anos', 'Inscritos com 22 anos',
       'Inscritos com 23 anos', 'Inscritos com 24 anos',
       'Inscritos com 25 anos', 'Inscritos entre 26 e 30 anos',
       'Inscritos entre 31 e 35 anos', 'Inscritos entre 36 e 40 anos',
       'Inscritos entre 41 e 45 anos', 'Inscritos entre 46 e 50 anos',
       'Inscritos entre 51 e 55 anos', 'Inscritos entre 56 e 60 anos',
       'Inscritos entre 61 e 65